In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import impala
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.01
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=500)
    env.update_taskset(TaskSet(preset=['C8']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.impala import ImpalaTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(ImpalaTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 2,
             "entropy_coeff": 0.01,
             "train_batch_size": 5000,
             "grad_clip": 100.0,
             "rollout_fragment_length": 200,
             # Number of passes to make over each train batch.
             "num_sgd_iter": 1,
             "num_multi_gpu_tower_stacks": 1,
             "replay_proportion": 0.0,
             "replay_buffer_num_slots": 100,
             "learner_queue_size": 32,
             "learner_queue_timeout": 3000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "IMPALA C32 pretrained (AnnaCNN) (3 noops after placement) and r: -0.01"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-10-20 16:42:14,635	INFO wandb.py:170 -- Already logged into W&B.
2021-10-20 16:42:14,648	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
IMPALA_my_env_ae052_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id c4d8f_00000 but id ae052_00000 is set.
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=5416) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=5416)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=5416) 2021-10-20 16:42:18,023	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=5410) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=5410)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=5414) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=5414)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=5416) 2021-10-20 16:42:23,780	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=5410) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=5410)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=5414) /root/miniconda/envs/py37/lib/

Result for IMPALA_my_env_ae052_00000:
  agent_timesteps_total: 5008
  custom_metrics: {}
  date: 2021-10-20_16-44-11
  done: false
  episode_len_mean: 212.0
  episode_media: {}
  episode_reward_max: -1.7400000000000013
  episode_reward_mean: -2.4526086956521707
  episode_reward_min: -6.939999999999987
  episodes_this_iter: 23
  episodes_total: 23
  experiment_id: 68f712a722a9472ba0c278ae092ebc50
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy: {}
    learner_queue:
      size_count: 1
      size_mean: 0.0
      size_quantiles:
      - 0.0
      - 0.0
      - 0.0
      - 0.0
      - 0.0
      size_std: 0.0
    num_agent_steps_sampled: 5008
    num_steps_sampled: 5008
    num_steps_trained: 1
    timing_breakdown:
      learner_dequeue_time_ms: 106968.159
      learner_grad_time_ms: 96.306
      learner_load_time_ms: 20.912
      learner_load_wait_time_ms: 106987.921
  iterations_since_restore: 1
  node_ip: 192.168.3.5
  num_healthy_workers: 2
  off_policy_estim

...

Result for IMPALA_my_env_ae052_00000:
  agent_timesteps_total: 590008
  custom_metrics: {}
  date: 2021-10-20_18-25-00
  done: false
  episode_len_mean: 186.41
  episode_media: {}
  episode_reward_max: -0.8500000000000005
  episode_reward_mean: -1.8641000000000012
  episode_reward_min: -2.159999999999998
  episodes_this_iter: 27
  episodes_total: 2476
  experiment_id: 68f712a722a9472ba0c278ae092ebc50
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy: {}
    learner_queue:
      size_count: 118
      size_mean: 0.0
      size_quantiles:
      - 0.0
      - 0.0
      - 0.0
      - 0.0
      - 0.0
      size_std: 0.0
    num_agent_steps_sampled: 590008
    num_steps_sampled: 590008
    num_steps_trained: 118
    num_weight_broadcasts: 116
    timing_breakdown:
      learner_dequeue_time_ms: 727695.771
      learner_grad_time_ms: 83.437
      learner_load_time_ms: 38.148
      learner_load_wait_time_ms: 62618.22
  iterations_since_restore: 118
  node_ip: 192.168.3.5

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
IMPALA_my_env_ae052_00000,RUNNING,192.168.3.5:5416,118,6153.81,590008,-1.8641,-0.85,-2.16,186.41


Result for IMPALA_my_env_ae052_00000:
  agent_timesteps_total: 595000
  custom_metrics: {}
  date: 2021-10-20_18-26-03
  done: false
  episode_len_mean: 186.05
  episode_media: {}
  episode_reward_max: -0.8500000000000005
  episode_reward_mean: -1.860500000000001
  episode_reward_min: -2.159999999999998
  episodes_this_iter: 26
  episodes_total: 2502
  experiment_id: 68f712a722a9472ba0c278ae092ebc50
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy: {}
    learner_queue:
      size_count: 119
      size_mean: 0.0
      size_quantiles:
      - 0.0
      - 0.0
      - 0.0
      - 0.0
      - 0.0
      size_std: 0.0
    num_agent_steps_sampled: 595000
    num_steps_sampled: 595000
    num_steps_trained: 119
    num_weight_broadcasts: 117
    timing_breakdown:
      learner_dequeue_time_ms: 727695.771
      learner_grad_time_ms: 83.282
      learner_load_time_ms: 38.148
      learner_load_wait_time_ms: 63684.714
  iterations_since_restore: 119
  node_ip: 192.168.3.5

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
IMPALA_my_env_ae052_00000,RUNNING,192.168.3.5:5416,119,6217.06,595000,-1.8605,-0.85,-2.16,186.05


(pid=5410) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=5410)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=5414) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=5414)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
